### Taking low dataset

* train_test_split
* data preprocessing
  

### Model building and test with mlflow and kfold

** Test with 3 dataset. 1 dataset for each models. **
- defog - and its models. (search for its best models.)
- tdcsfog - and its models. (search for its best models and parameters)
- Both dataset combine and its models. (search for its best models and parameters)

### Model searching the best parameter

### Testing with many data again and Retune

### final model with test data


In [1]:
import pandas as pd

set_defog_dataset = pd.read_parquet('Data/Process/set_defog_data.parquet')
# test_defog_dataset = pd.read_parquet('Data/Process/test_defog_data.parquet')
# Test_defog_dataset = pd.read_csv('Data/test/defog/02ab235146.csv')

In [2]:
def preprocessing_the_dataset(df):
    def check_skewness(df):
    # this can check relation between each column
        skew_limit=0.75
        skew_value=df[df.columns].skew()
        #print(skew_value)
        skew_col=skew_value[abs(skew_value)>skew_limit]
        cols=skew_col.index
        return cols

    import random 
    random_seed = 54
    
    feature_col = ['AccV','AccML','AccAP']
    label_col = ['StartHesitation','Turn','Walking', 'All_zero']

    # make feature and label
    feature_dataset = df[feature_col]
    label_dataset = df[label_col]
    
    # check skewness and powertransform
    skew_columns = check_skewness(feature_dataset)
    print(skew_columns)
    
    print("Power Transform start")
    from sklearn.preprocessing import PowerTransformer
    pt=PowerTransformer(standardize=False)  
    feature_dataset[skew_columns] = pt.fit_transform(feature_dataset[skew_columns])
    
    print("Standardization start")
    # Change features data to 0 and 1
    from sklearn.preprocessing import StandardScaler
    sc=StandardScaler()
    feature_dataset=sc.fit_transform(feature_dataset)
    
    print("Train test split begin")
    from sklearn.model_selection import train_test_split
    train_feature, valid_feature, train_label, valid_label = train_test_split(feature_dataset, label_dataset, test_size=0.2, random_state=True)
    
    return train_feature, valid_feature, train_label, valid_label
    

In [3]:
train_feature, valid_feature, train_label, valid_label = preprocessing_the_dataset(set_defog_dataset)

Index(['AccAP'], dtype='object')
Power Transform start


In [ ]:
train_feature.shape, valid_feature.shape, train_label.shape, valid_label.shape

### Build Model

In [ ]:
# SKlearn model
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import  RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import NuSVC
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from catboost import Pool, CatBoostClassifier, cv
# import lightgbm as lgb
import xgboost as xgb

In [ ]:


#Building Model Dict
Models = {
    "Logistic Regression": LogisticRegression(),                    
    "Support Vector Classifier": SVC(),                             
    "Decision Tree": DecisionTreeClassifier(),           
    "KNearest": KNeighborsClassifier(),                
    "GaussianNB" : GaussianNB(),                                    
    "LDA" : LinearDiscriminantAnalysis(),                            
    "Ridge" : RidgeClassifier(),                                      
    "QDA" : QuadraticDiscriminantAnalysis(),                        
    "Bagging" : BaggingClassifier(),                                
    "MLP" : MLPClassifier(),                                        
    "LSVC" : LinearSVC(),                                           
    "BernoulliNB" : BernoulliNB(),                                  
    "Passive_AC" : PassiveAggressiveClassifier(),                   
    "SGB"     : GradientBoostingClassifier(),
    "Adaboost" : AdaBoostClassifier(),
    "Extra_T" : ExtraTreesClassifier(),
    "R_forest" : RandomForestClassifier(),
    "XGB" : xgb.XGBClassifier(),
    "Catboost" : CatBoostClassifier()}



In [ ]:
# # LGB

# lgb_train = lgb.Dataset(train_nm_features, train_nm_labels,  params={'verbose': -1})
# lgb_test = lgb.Dataset(train_nm_features, train_nm_labels,  params={'verbose': -1}, reference=lgb_train)

# LGBM = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_test], verbose_eval=False)
# LGBM_pred=LGBM.predict(test_nm_features)
# LGBM_pred = LGBM_pred.round()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import precision_score,confusion_matrix,classification_report

In [ ]:
np.random.seed(40)
from mlflow import MlflowClient
from mlflow.entities import ViewType
from mlflow.models.signature import infer_signature

!mlflow server --backend-store-uri sqlite:///backend.db

In [ ]:
def eval_metrics(classifier, test_features, test_labels):
    
    # make prediction
    predictions   = classifier.predict(test_features)
    
    base_score   = classifier.score(test_features,test_labels)
    accuracy = accuracy_score(test_labels, predictions)
    av_precision = average_precision_score(test_labels, predictions)
    precision = precision_score(test_labels, predictions)
    recall = recall_score(test_labels, predictions)
    f1score = f1_score(test_labels, predictions)
    Matrix = confusion_matrix(test_labels, predictions)
    
    matrix_scores = { 
        "true negative"  : Matrix[0][0],
        "false positive" : Matrix[0][1],
        "false negative" : Matrix[1][0],
        "true positive " : Matrix[1][1]
    }
    
    target_names = ['0','1']
    print("Classification report")
    print("---------------------","\n")
    print(classification_report(test_labels, predictions,target_names=target_names),"\n")
    print("Confusion Matrix")
    print("---------------------","\n")
    print(f"{Matrix} \n")

    print("Accuracy Measures")
    print("---------------------","\n")
    print("Base score: ", base_score)
    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1score)
    
    return base_score,accuracy,precision,recall,f1score,matrix_scores

In [ ]:


# # This function train all the Dataset and all models
# def Train_all_Model():

#     Dataset_scores = {}

#     for Name, values in Meta_Dataset.items():
#         print(f"The {Name} Dataset is using.........")
#         i = 1
#         Model_scores = {} #Model = Socre
#         for Model_Name, classifier in Models.items():
#             print(f"{i}. {Model_Name}")
#             classifier.fit(values[0], values[1])
#             score = calculate_socre(classifier=classifier,
#                                     test_features=values[2],
#                                     test_labels=values[3])
#             i = i+1
#             print(f"{score}")
#             Model_scores[Model_Name] = score
#         print("________________________________________")
#         Dataset_scores[Name] = Model_scores
#     return Dataset_scores



In [ ]:
counter = 1
for Model_Name, classifier in Models.items(): 
    # with mlflow.start_run(nested=True):
    print(f"{counter}. {Model_Name}")
    
    with mlflow.start_run():
        # fit the model
        classifier.fit(train_feature, train_label)
        
        counter = counter + 1
        
        # Calculate the metrics
        base_score,accuracy,precision,recall,f1score,matrix_scores = eval_metrics(classifier,
                                                                                  valid_feature,
                                                                                  valid_label)  
        
        mlflow.log_param("Model"           , Model_Name)
        mlflow.log_param("Sampling"        , sampling)
        mlflow.log_metric("base_score"     , base_score)
        mlflow.log_metric("accuracy"       , accuracy)
        mlflow.log_metric("av_precision"   , precision)
        mlflow.log_metric("recall"         , recall)
        mlflow.log_metric("f1"             , f1score)
        mlflow.log_params(matrix_scores)
        
        signature = infer_signature(test_features, classifier.predict(test_features))
        if f1score > 0.95 :
            mlflow.sklearn.log_model(classifier,Model_Name, signature=signature)
            print(f"f1 socre is more than 0.945 so the {Model_Name} is saved")
        else :
            print(f"Because f1 socre is not quality. The model is skip to saving phase.")
        
        print("________________________________________")